Priprema podataka
- dobivanje parova nizova iz poravnanja višestrukih sekvenci (engl. Multiple sequence alignment)
- korištenje podataka ([HIV Sequence Alignments](https://www.hiv.lanl.gov/content/sequence/NEWALIGN/align.html)) za inicijalizaciju matrica u Baum-Welch algoritmu


In [1]:
from tqdm import tqdm
import itertools
from collections import Counter

In [11]:
from tqdm import tqdm

def read_fasta(file_path):
    sequences = []
    current_sequence = ""
    current_header = ""

    total_lines = sum(1 for line in open(file_path, 'r'))

    with open(file_path, 'r') as file:
        for line in tqdm(file, total=total_lines, desc="Reading FASTA file"):
            line = line.strip()
            if line.startswith('>'):
                if current_header: #dodaj sekvencu
                    sequences.append(current_sequence)
                current_header = line[1:]
                current_sequence = ""
            else:
                current_sequence += line

    #zadnja sekvenca
    if current_header:
        sequences.append(current_sequence)

    return sequences


fasta_file_path = 'HIV1_ALL_2021_genome_DNA.fasta'
#trebalo bi učitati 1000 sekvenci ili barem 500
sequences = read_fasta(fasta_file_path)[:500]#radi ograničenih resursa zadržat ćemo 500 sekvenci





Reading FASTA file: 100%|████████████████████████████████████████████████| 1515900/1515900 [00:02<00:00, 709162.99it/s]


In [12]:
print(f"Ukupan broj sekvenci: {len(sequences)}")
print(f"Duljina sekvenci: {Counter([len(i) for i in sequences])}")

Ukupan broj sekvenci: 500
Duljina sekvenci: Counter({14937: 500})


In [16]:
#napravi sve parove od poravnatih sekvenci (i (x,y) i (y,x))
pairs = list(itertools.permutations(sequences, 2))
len(pairs)

249500

In [17]:
#parovi = []
stanja = {}
stanja_lista = []


for pair in tqdm(pairs):
    
    prvi = ''.join(a  for a, b in zip(pair[0], pair[1]) if a != '-' or b != '-' )
    drugi = ''.join(b for a, b in zip(pair[0], pair[1]) if a != '-' or b != '-' )


    novi_prvi = ''.join('' if ((prvi[i] == '-' and i > 0 and drugi[i-1] == '-') or (prvi[i] == '-' and i < (len(drugi) - 1) and drugi[i + 1] == '-')) else prvi[i] for i in range(len(prvi)))
    novi_drugi = ''.join('' if ((drugi[i] == '-' and i > 0 and prvi[i-1] == '-') or (drugi[i] == '-' and i < (len(prvi) - 1) and prvi[i + 1] == '-')) else drugi[i] for i in range(len(drugi)))
    
    pair2 = (novi_prvi,novi_drugi)
    
    new_pair = [
        'M' if a != '-' and b != '-' else
        'Ix' if a != '-' and b == '-' else
        'Iy'
        for a, b in zip(pair2[0], pair2[1])
    ]

    stanja[pair2] = new_pair
    stanja_lista.append(new_pair)
    

    #parovi.append((novi_prvi,novi_drugi))


 43%|████████████████████████████████                                          | 108228/249500 [25:28<33:15, 70.78it/s]


MemoryError: 

In [4]:
#len(stanja_lista)

In [5]:
#parovi[0][1]

In [6]:
#list(itertools.permutations([1,2,3],2))

In [7]:
#list(itertools.combinations([1,2,3],2))

In [8]:
#ovo za 1% traje 30 minuta kad se radi na svim podatcima
#pairs2 = []
#duljina = len(pairs[0][0])  # sve sekvence su jednake duljine

#for pair in tqdm(pairs):
#    new_pair = (''.join(a for a, b in zip(pair[0], pair[1]) if a != '-' or b != '-'),
#        ''.join(b for a, b in zip(pair[0], pair[1]) if a != '-' or b != '-')
#    )

#    pairs2.append(new_pair)




In [9]:
def preprocess(lista_parova):
    rezultat = []

    for pair1, pair2 in tqdm(lista_parova):
        prvi = ''.join(a for a, b in zip(pair1, pair2) if a != '-' or b != '-')
        drugi = ''.join(b for a, b in zip(pair1, pair2) if a != '-' or b != '-')
       
        
        novi_prvi = ""
        novi_drugi = ""

        
        for i, znak in enumerate(prvi):
            if (znak == '-' and i > 0 and drugi[i - 1] == '-') or (znak == '-' and i < (len(drugi) - 1) and drugi[i + 1] == '-'):
                continue
            novi_prvi += znak
            
        for i, znak in enumerate(drugi):
            if (znak == '-' and i > 0 and prvi[i - 1] == '-') or (znak == '-' and i < (len(prvi) - 1) and prvi[i + 1] == '-'):
                continue
            novi_drugi += znak

        rezultat.append((novi_prvi, novi_drugi))

    return rezultat

#pairs2 = preprocess(pairs)

In [9]:
#za svaki par treba parsirati susjedne procjepe ('-AC'i 'T-C' postaje 'AC' i 'TC')
#ako imamo Ix pa Iy ili Iy pa Ix to postaje M

In [15]:
#za svaki par napravi string sa stanjima - Ix,Iy,M

#stanja = {}
#stanja_lista = []
#for pair in tqdm(pairs2):
#    new_pair = [
#        'M' if a != '-' and b != '-' else
#        'Ix' if a != '-' and b == '-' else
#        'Iy'
#        for a, b in zip(pair[0], pair[1])
#    ]
    
#    stanja[pair] = new_pair
#    stanja_lista.append(new_pair)



 44%|███████████████████████████████▉                                         | 109178/249500 [03:31<04:32, 515.03it/s]


MemoryError: 

In [10]:
len(stanja)

NameError: name 'stanja' is not defined

In [25]:
prvi_element = [i[0] for i in stanja_lista]
broj_M = prvi_element.count('M')
broj_Ix = prvi_element.count('Ix')
broj_Iy = prvi_element.count('Iy')
print(broj_M,broj_Ix,broj_Iy)


7 93 0


#### matrica početnih vjerojatnosti $\pi$

In [29]:
#0-index je M, 1-index je Ix, 2-index je Iy

pi = [broj_M/len(prvi_element), broj_Ix/len(prvi_element), broj_Iy/len(prvi_element)]
#ako slucajno bude 0 to treba zamjeniti s malom vrijednošću
eps = 0.00001
pi = [max(vjerojatnost,eps) for vjerojatnost in pi]
pi

[0.07, 0.93, 1e-05]

#### matrica vjerojatnosti prijelaza A

In [49]:
import numpy as np

stanja_set = ['M', 'Ix', 'Iy']

#inicijalizacija
matrica_prijelaza = np.zeros((len(stanja_set), len(stanja_set)))

#frekvencije
for par in stanja_lista:
    for i in range(len(par) - 1):
        trenutno_stanje = stanja_set.index(par[i])
        sljedece_stanje = stanja_set.index(par[i + 1])
        matrica_prijelaza[trenutno_stanje, sljedece_stanje] += 1

#normalizacija
matrica_prijelaza = matrica_prijelaza/np.sum(matrica_prijelaza, axis=1, keepdims=True)

matrica_prijelaza

array([[0.99723469, 0.00166448, 0.00110083],
       [0.0140742 , 0.9859258 , 0.        ],
       [0.11846992, 0.        , 0.88153008]])

In [50]:
#zamijeniti nule s epsilonom
matrica_prijelaza = np.where(matrica_prijelaza == 0, eps, matrica_prijelaza)
matrica_prijelaza

array([[9.97234690e-01, 1.66447727e-03, 1.10083258e-03],
       [1.40741970e-02, 9.85925803e-01, 1.00000000e-05],
       [1.18469918e-01, 1.00000000e-05, 8.81530082e-01]])

#### matrica emisija E

In [ ]:
#emitirati se mogu parovi AA,AC,AG,AT,CC,CA,CG,CT,GG,GA,GC,GT,TT,TA,TC,TG,A,C,G,T,-(ne znam trebamo li ga kad obrišemo -- slučaj)
#stanja M,Ix,Iy

In [54]:
emisija = sorted(["AA","AC","AG","AT","CC","CA","CG","CT","GG","GC","GT","GA","TT","TA","TC","TG","A","G","C","T"])

In [59]:
#primjer = {("AA-","TGA"):["M","M","Iy"],("AC-","C-A"):["M","Ix","Iy"]}
#primjer

{('AA-', 'TGA'): ['M', 'M', 'Iy'], ('AC-', 'C-A'): ['M', 'Ix', 'Iy']}

In [63]:
matrica_emisija = np.zeros((len(stanja_set), len(emisija)))


In [64]:
for par, stanja in stanja.items():
    for i, simbol_stanja in enumerate(stanja):
        x,y = par
        if x[i] != '-' and y[i] != '-':
            par_simbola = x[i] + y[i]
        else:
            if x[i] == '-':
                par_simbola = y[i]
            else:
                par_simbola = x[i]
        print(par_simbola,simbol_stanja)
        matrica_emisija[stanja_set.index(simbol_stanja), emisija.index(par_simbola)] += 1
matrica_emisija

AT M
AG M
A Iy
AC M
C Ix
A Iy


array([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [67]:
matrica_emisija = matrica_emisija/np.sum(matrica_emisija, axis=1, keepdims=True)#zbroj po retcima

matrica_emisija

array([[0.        , 0.        , 0.33333333, 0.33333333, 0.33333333,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [68]:
#zamijeniti nule s epsilonom
matrica_emisija = np.where(matrica_emisija == 0, eps, matrica_emisija)
matrica_emisija

array([[1.00000000e-05, 1.00000000e-05, 3.33333333e-01, 3.33333333e-01,
        3.33333333e-01, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05],
       [1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e+00, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05],
       [1.00000000e+00, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.00000000e-05,
        1.00000000e-05, 1.00000000e-05, 1.00000000e-05, 1.0000